In [58]:
"""
CommandCenter File
read from influx and publish to mqtt broker
"""

# Importing relevant modules
import os
from dotenv import load_dotenv
from influxdb_client import InfluxDBClient, Point,  Dialect
from influxdb_client.client.write_api import ASYNCHRONOUS
import paho.mqtt.client as mqtt
import time
import json
import requests
import pandas as pd

# Load environment variables from ".env"
load_dotenv()

# InfluxDB config
BUCKET = 'fullstack-influxdb' #  bucket is a named location where time series data is stored.
url = 'https://iot-group2-service1.iotcloudserve.net/'
token ='Dwj0HPIYScc1zvkB0zHpjxIVIssU_z_-unniio7sOcZl135FZ40ONj9ZX6jgiBWqkwpOQegRAL21Ix1z86SBJw=='
org = 'Chulalongkorn'
print("connecting to",url)

client = InfluxDBClient(

    url= url,
    token= token,
    org= 'Chulalongkorn'
)
write_api = client.write_api()
query_api = client.query_api()

# MQTT broker config
MQTT_BROKER_URL = "172.20.10.3"
MQTT_SUBSCRIBE_TOPIC = "@msg/datacc2broker"
print("connecting to MQTT Broker", MQTT_BROKER_URL)
mqttc = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
mqttc.connect(MQTT_BROKER_URL,1883)

# read from influx and pub to mqtt
def on_connect(client, userdata, flags, rc, properties):
    """ The callback for when the client connects to the broker."""
    print("Connected with result code "+str(rc))

# def extract_data():
query = 'from(bucket:"fullstack-influxdb")\
|> range(start: -12h)\
|> filter(fn:(r) => r._measurement == "prediction_data")\
|> filter(fn:(r) => r._field == "try8")'


csv_result = query_api.query_csv(query,
                                dialect=Dialect(header=False, delimiter=",", comment_prefix="#", annotations=[],
                                                date_time_format="RFC3339"))
df = pd.DataFrame(csv_result)
print(csv_result)


connecting to https://iot-group2-service1.iotcloudserve.net/
connecting to MQTT Broker 172.20.10.3


In [59]:
df.head()

,0,1,2,3,4,5,6,7,8
0,,_result,0,2024-05-13T02:00:22.054145775Z,2024-05-13T14:00:22.054145775Z,2024-05-13T13:10:55.035618127Z,37.02671813964844,try8,prediction_data
1,,_result,0,2024-05-13T02:00:22.054145775Z,2024-05-13T14:00:22.054145775Z,2024-05-13T13:11:06.143766028Z,36.999061584472656,try8,prediction_data
2,,_result,0,2024-05-13T02:00:22.054145775Z,2024-05-13T14:00:22.054145775Z,2024-05-13T13:11:14.96705421Z,36.90504837036133,try8,prediction_data
3,,_result,0,2024-05-13T02:00:22.054145775Z,2024-05-13T14:00:22.054145775Z,2024-05-13T13:11:24.195749263Z,36.86664962768555,try8,prediction_data
4,,_result,0,2024-05-13T02:00:22.054145775Z,2024-05-13T14:00:22.054145775Z,2024-05-13T13:11:33.074598524Z,36.90504837036133,try8,prediction_data


In [61]:
columns_to_keep = [5, 6]
df = df[df.columns[columns_to_keep]]
df

,5,6
0,2024-05-13T13:10:55.035618127Z,37.02671813964844
1,2024-05-13T13:11:06.143766028Z,36.999061584472656
2,2024-05-13T13:11:14.96705421Z,36.90504837036133
3,2024-05-13T13:11:24.195749263Z,36.86664962768555
4,2024-05-13T13:11:33.074598524Z,36.90504837036133
5,2024-05-13T13:11:43.488908203Z,36.924076080322266
6,2024-05-13T13:12:15.92943551Z,36.9042854309082
7,2024-05-13T13:12:27.281622882Z,36.9420166015625
8,2024-05-13T13:12:36.103748691Z,36.98854446411133
9,2024-05-13T13:12:45.892699834Z,36.97929382324219


In [62]:
new_column_names = ['time','temp_bmp280']

# Rename the columns
df = df.rename(columns=dict(zip(df.columns, new_column_names)), inplace=False)
df

,time,temp_bmp280
0,2024-05-13T13:10:55.035618127Z,37.02671813964844
1,2024-05-13T13:11:06.143766028Z,36.999061584472656
2,2024-05-13T13:11:14.96705421Z,36.90504837036133
3,2024-05-13T13:11:24.195749263Z,36.86664962768555
4,2024-05-13T13:11:33.074598524Z,36.90504837036133
5,2024-05-13T13:11:43.488908203Z,36.924076080322266
6,2024-05-13T13:12:15.92943551Z,36.9042854309082
7,2024-05-13T13:12:27.281622882Z,36.9420166015625
8,2024-05-13T13:12:36.103748691Z,36.98854446411133
9,2024-05-13T13:12:45.892699834Z,36.97929382324219


In [63]:
import pytz
# Convert 'time' column to datetime format
df['time'] = pd.to_datetime(df['time'])

# Convert 'time' column to Thailand timezone
thailand_tz = pytz.timezone('Asia/Bangkok')
df['time'] = df['time'].dt.tz_convert(thailand_tz)

# Format 'time' column as desired
df['time'] = df['time'].dt.strftime('%Y/%m/%d %H:%M:%S')

In [64]:
df

,time,temp_bmp280
0,2024/05/13 20:10:55,37.02671813964844
1,2024/05/13 20:11:06,36.999061584472656
2,2024/05/13 20:11:14,36.90504837036133
3,2024/05/13 20:11:24,36.86664962768555
4,2024/05/13 20:11:33,36.90504837036133
5,2024/05/13 20:11:43,36.924076080322266
6,2024/05/13 20:12:15,36.9042854309082
7,2024/05/13 20:12:27,36.9420166015625
8,2024/05/13 20:12:36,36.98854446411133
9,2024/05/13 20:12:45,36.97929382324219


In [57]:

# for index, value in df3['temp_bmp280'].iteritems():
#     # publish the data to MQTT Broker
#     datastr = f" temp_bmp280: {value}"
#     result = mqttc.publish(MQTT_SUBSCRIBE_TOPIC, datastr)
#     status = result[0]
#     if status == 0:
#         print(f"Send `{datastr}` to topic `{MQTT_SUBSCRIBE_TOPIC}`")
#     else:
#         print(f"Failed to send message to topic {MQTT_SUBSCRIBE_TOPIC}")
# for value in df3['temp_bmp280'].items():
#     # publish the data to MQTT Broker
#     datastr = f"temp_bmp280: {value[1]}"
#     result = mqttc.publish(MQTT_SUBSCRIBE_TOPIC, datastr)
#     status = result[0]
#     if status == 0:
#         print(f"Send `{datastr}` to topic `{MQTT_SUBSCRIBE_TOPIC}`")
#     else:
#         print(f"Failed to send message to topic {MQTT_SUBSCRIBE_TOPIC}")

datastr = f"temp_bmp280: {df['temp_bmp280'].iloc[-1]}"
result = mqttc.publish(MQTT_SUBSCRIBE_TOPIC, datastr)
status = result[0]
if status == 0:
    print(f"Send `{datastr}` to topic `{MQTT_SUBSCRIBE_TOPIC}`")
else:
    print(f"Failed to send message to topic {MQTT_SUBSCRIBE_TOPIC}")

        

        # mqttc.publish(MQTT_PUBLISH_TOPIC, json.dumps(payload))
        # print(f"Published new measurement: {json.dumps(payload)}")

    # #csv_result =  query_api.query_csv(query2)
    # df = pd.DataFrame(csv_result)
    # print("Successfully convert to DataFrame")
    # # print(df.iloc[:-5])
    # #df.to_csv('RaspberryPi/CommandCenter/12_5_12_50.csv', index=False)

    # client.close()

    # columns_to_keep = [5, 7, 8, 9, 10]
    # df = df[df.columns[columns_to_keep]]

    # new_column_names = ['time', 'humid_sht4x', 'pressure_bmp280', 'temp_bmp280', 'temp_sht4x']

    # df = df.rename(columns=dict(zip(df.columns, new_column_names)), inplace=False)

    # df['time'] = pd.to_datetime(df['time'])
    # # Format 'time' column as desired
    # df['time'] = df['time'].dt.strftime('%Y/%m/%d %H:%M:%S')
    # df.head()


mqttc.on_connect = on_connect
time.sleep(1)
#mqttc.loop_forever()

# # Main loop
# while True:
#     extract_data()
#     time.sleep(10)  # Sleep for 10 seconds before the next extraction


Send `temp_bmp280: 36.94148254394531` to topic `@msg/datacc2broker`
